In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.5 MB/s eta 0:00:00


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 10.0 MB/s eta 0:00:00


## Generating prompts to feed into the model

In [3]:
from datasets import load_dataset

example = 2

imdb = load_dataset("imdb", split="train+test")
prompts = [" ".join(review.split()[:3]) for review in imdb["text"]]
review_text = [" ".join(review.split()[3:]) for review in imdb["text"]]
print(f"Prompt: {prompts[4]} \nDataset continuation: {review_text[2]} \nLabel: {imdb['label'][3]}")


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.
Prompt: Oh, brother...after hearing 
Dataset continuation: avoid making this type of film in the future. This film is interesting as an experiment but tells no cogent story.<br /><br />One might feel virtuous for sitting thru it because it touches on so many IMPORTANT issues but it does so without any discernable motive. The viewer comes away with no new perspectives (unless one comes up with one while one's mind wanders, as it will invariably do during this pointless film).<br /><br />One might better spend one's time staring out a window at a tree growing.<br /><br /> 
Label: 0


## Generating text using a GPT-2 model

The model is finetuned on the IMDB dataset for one epoch and produces both positive and negative reviews.

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("lvwerra/gpt2-imdb")

model = AutoModelForCausalLM.from_pretrained("lvwerra/gpt2-imdb")

inputs = tokenizer(prompts[3], return_tensors='pt')
gpt2_outputs = tokenizer.decode(model.generate(**inputs, do_sample=True, top_p=1 , max_new_tokens=40).squeeze(0))
gpt2_outputs

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'This film was pretty dull, especially the death scene, the entire film is shot the same way, the director obviously missed what they had planned. I have to say the ending really is disappointing though the only reason to'

## Reward function

We are using a BERT model fine-tuned on the IMDB dataset to act as our text sentiment classifier. This classifier will be used to give positive signals to GPT-2 when it produces positive text and vice versa. 

In [5]:
import torch
from transformers import pipeline
from typing import List

device = 'cuda' if torch.cuda.is_available() else 'cpu'

sentiment_model = pipeline(
        "sentiment-analysis",
        "lvwerra/distilbert-imdb",
        top_k=2,
        truncation=True,
        batch_size=256,
        device=device,
    )

def reward_fn(samples: List[str], **kwargs) -> List[float]:
    sentiments = list(map(get_positive_score, sentiment_model(samples)))
    return sentiments

def get_positive_score(scores):
    "Extract value associated with a positive sentiment from pipeline's output"
    return dict(map(lambda x: tuple(x.values()), scores))["POSITIVE"]


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


## Training Loop

In [6]:
!git clone https://github.com/CarperAI/trlx.git
%cd trlx
!pip install torch==2.0.0 --extra-index-url https://download.pytorch.org/whl/cu116 # for cuda
!pip install -e .

Cloning into 'trlx'...
remote: Enumerating objects: 7207, done.
remote: Counting objects: 100% (2454/2454), done.
remote: Compressing objects: 100% (471/471), done.
remote: Total 7207 (delta 2152), reused 2114 (delta 1983), pack-reused 4753
Receiving objects: 100% (7207/7207), 46.67 MiB | 20.06 MiB/s, done.
Resolving deltas: 100% (4871/4871), done.
/content/trlx
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.2 MB/s eta 0:00:00
     ━━━━━

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/trlx
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 41.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━

In [7]:
import trlx
from trlx.data.default_configs import TRLConfig, default_ppo_config

hparams = {}
config = TRLConfig.update(default_ppo_config().to_dict(), hparams)

trlx.train(
        reward_fn=reward_fn,
        prompts=prompts,
        eval_prompts=["I don't know much about Hungarian underground"] * 256,
        config=config,
    )


Setting ds_accelerator to cuda (auto detect)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 7>:7                                                                              │
│                                                                                                  │
│ /content/trlx/trlx/trlx.py:79 in train                                                           │
│                                                                                                  │
│    76 │   if model_path:                                                                         │
│    77 │   │   config.model.model_path = model_path                                               │
│    78 │                                                                                          │
│ ❱  79 │   trainer = get_trainer(config.train.trainer)(                                           │
│    80 │   │   config=config,                                                                     │
│    81 │   │   reward_fn=reward_fn,                                                               │
│    82 │   │   metric_fn=metric_fn,                                                               │
│                                                                                                  │
│ /content/trlx/trlx/trainer/accelerate_ppo_trainer.py:46 in __init__                              │
│                                                                                                  │
│    43 │   │   Args:                                                                              │
│    44 │   │   │   config: Config                                                                 │
│    45 │   │   """                                                                                │
│ ❱  46 │   │   super().__init__(config, **kwargs)                                                 │
│    47 │   │                                                                                      │
│    48 │   │   # Setup rollout logging                                                            │
│    49 │   │   if config.train.rollout_logging_dir is not None:                                   │
│                                                                                                  │
│ /content/trlx/trlx/trainer/accelerate_base_trainer.py:58 in __init__                             │
│                                                                                                  │
│    55 │   │   │   self.mb_size = config.train.batch_size                                         │
│    56 │   │   self.num_mb = config.train.batch_size // self.mb_size                              │
│    57 │   │   self.mb_count = 0                                                                  │
│ ❱  58 │   │   self.accelerator = Accelerator(log_with=config.train.tracker, logging_dir=config   │
│    59 │   │                                                                                      │
│    60 │   │   if self.accelerator.state.deepspeed_plugin is not None:                            │
│    61 │   │   │   # by accelerate's default, arguments in `model.forward` would be casted to h   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: Accelerator.__init__() got an unexpected keyword argument 'logging_dir'